In [1]:
"""
프로젝트 진행 기본 코드
"""

import pandas as pd
import datetime as dt


# 날짜 차이 계산
def calculate_days_difference(date):
    NOW = dt.datetime(2022, 12, 1)
    # 현재 날짜와 입력 날짜 간의 차이를 계산
    difference = NOW - date
    return difference.days


# 데이터 파일 불러오기
def read_data(path):
    df = pd.read_csv(path)

    # 주문일자 컬럼 datetime 자료형으로 변환
    df["주문일자"] = pd.to_datetime(df["주문일자"])

    return df


# 인자로 전달한 dataframe을  csv 파일로 저장
def save_dataframe(dataframe: pd.DataFrame):
    dataframe.to_csv("RFM_result.csv")


file_path = "shopping.csv"

df = read_data(file_path)

# print(df.head())

# print(df.isnull().sum()) # 결측치 확인

new_df = pd.DataFrame()

df = read_data(file_path)  # 주문일자 날짜데이터로 변환

group = df.groupby("구매자")  # 구매자 기준으로 그룹화

g = group["주문일자"].max()  # 그룹의 주문일자의 최댓값들만 추출

new_df["구매자"] = g.index
new_df["recency"] = g.values  # 각각 할당

f = df["구매자"].value_counts()  # 같은 사람이 산 상품 기록 개수 계산

new_df = pd.merge(
    new_df, f, how="inner", on="구매자"
)  # 두 데이터프레임 구매자 기준으로 합치기

new_df = new_df.rename({"count": "frequency"}, axis=1)  # 열 이름 변환

r = group["판매금액"].sum()  # 구매자당 판매금액 합 계산

new_df = pd.merge(
    new_df, r, how="inner", on="구매자"
)  # 데이터프레임에 구매자 기준으로 합치기

new_df = new_df.rename({"판매금액": "monetary"}, axis=1)  # 열 이름 변환

for row in new_df.itertuples():  # 데이터프레임 순회하면서 R_score 저장
    res = calculate_days_difference(row.recency)
    if res <= 30:
        new_df.loc[row[0], "R_score"] = 5
    elif res <= 90:
        new_df.loc[row[0], "R_score"] = 4
    elif res <= 180:
        new_df.loc[row[0], "R_score"] = 3
    elif res <= 365:
        new_df.loc[row[0], "R_score"] = 2
    elif res > 365:
        new_df.loc[row[0], "R_score"] = 1

new_df["F_score"] = pd.qcut(
    new_df["frequency"], q=[0, 0.3, 0.5, 0.7, 0.9, 1.0], labels=[1, 2, 3, 4, 5]
)  # f score qcut 이용하여 저장

new_df["M_score"] = pd.qcut(
    new_df["monetary"], q=[0, 0.3, 0.5, 0.7, 0.9, 1.0], labels=[1, 2, 3, 4, 5]
)  # m score qcut 이용하여 저장

for row in new_df.itertuples():
    if new_df.loc[row[0], "R_score"] < 3:
        new_df.loc[row[0], "grade"] = "이탈 가능 고객"
    elif new_df.loc[row[0], "F_score"] < 3:
        new_df.loc[row[0], "grade"] = "신규 고객"
    elif (
        new_df.loc[row[0], "R_score"] == 5
        and new_df.loc[row[0], "F_score"] == 5
        and new_df.loc[row[0], "M_score"] == 5
    ):
        new_df.loc[row[0], "grade"] = "VIP"
    elif new_df.loc[row[0], "M_score"] > 3:
        new_df.loc[row[0], "grade"] = "고가치 고객"
    else:
        new_df.loc[row[0], "grade"] = "일반 고객"  # 점수에 따른 고객 유형 분류

save_dataframe(new_df)  # 데이터 csv로 저장

new_df

,구매자,recency,frequency,monetary,R_score,F_score,M_score,grade
0,Adams and Sons,2022-05-17 15:32:06,122,6834350,2.0,3,3,이탈 가능 고객
1,Adams-Adams,2022-11-04 11:11:05,83,925700,5.0,2,2,신규 고객
2,"Anderson, Fields and Harrison",2022-06-22 15:47:31,27,24392730,3.0,1,4,신규 고객
3,"Barker, Scott and Sanchez",2022-07-12 14:24:45,7,5030100,3.0,1,2,신규 고객
4,Blair-Cummings,2022-10-05 13:28:30,363,28435000,4.0,4,4,고가치 고객
...,...,...,...,...,...,...,...,...
84,Wilkins Ltd,2022-11-01 18:07:14,168,55596200,5.0,3,4,고가치 고객
85,Wilson Group,2021-04-08 16:53:06,31,4840000,1.0,2,2,이탈 가능 고객
86,Wise Group,2021-04-01 21:38:58,70,6004400,1.0,2,2,이탈 가능 고객
87,Young Inc,2022-11-08 01:57:09,3688,41057164,5.0,5,4,고가치 고객
